# FEM Simulatons of Navier - Stokes Flow in Cylindrical Cavity

Simulates Stokes flow in a cylindrical cavity. Code copied from [Ferrite.jl Navier-Stokes Flow Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/ns_vs_diffeq/). 

<b>Boundary conditions</b>: one can pass a vertex set containing a single vertex to a Dirichlet condition. 

<b>Questions</b>: 
1. how to read a point set from a mesh file generated by GMSH? In current attempts, the function getnodes() fails to recognize the key. Are there any examples that demonstrate the use of the getnodes() function or any of its equivalents? Can all the keys in the mesh file be retrieved?
2. why does automatic differentiation fails?
3. can explicit time integration methods be applied in testing phase.

<b>Relevant discussions on Ferrite.jl</b>
1. In case of using constraints (e.g. constraints on the pressure) it is required to pass the ch to allocate_matrix to get the correct sparsity pattern. See also [github](https://github.com/Ferrite-FEM/Ferrite.jl/discussions/1167). 

##  Import Packages

In [1]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

In [9]:
timestepper = Rodas5P(autodiff=false, step_limiter! = ferrite_limiter!);

LoadError: MethodError: no method matching Rodas5P(; autodiff::Bool, step_limiter!::typeof(ferrite_limiter!))

[0mClosest candidates are:
[0m  Rodas5P(; chunk_size, autodiff, standardtag, concrete_jac, diff_type, linsolve, precs)[91m got unsupported keyword argument "step_limiter!"[39m
[0m[90m   @[39m [35mOrdinaryDiffEq[39m [90m~/.julia/packages/OrdinaryDiffEq/NBaQM/src/[39m[90m[4malgorithms.jl:2958[24m[39m


## Section 1: Introduction 

Simulates Navier-Stokes flow in a cylindrical cavity. Code copied from [Ferrite.jl Navier-Stokes Flow Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/ns_vs_diffeq/). Requires more details.

## Section 2: Read 2D Mesh From External Input Mesh File  

In [41]:
grid = togrid("mesh_cavity.msh")

Info    : Reading 'mesh_cavity.msh'...
Info    : 9 entities
Info    : 10557 nodes
Info    : 59372 elements
Info    : Done reading 'mesh_cavity.msh'


Grid{3, Tetrahedron, Float64} with 48182 Tetrahedron cells and 10557 nodes

In [42]:
#getfacetset(grid,"wall")

## Section 3: Functions for Assembly of Mass Matrix, Stiffness Matrix and Load Vector 
Modified tutorial by removing the volumetric source term. 

In [43]:
function lid_rotation(X, t)
    # velocity in cylindrical coordinates (0, om, 0)
    x, y, z = X
    r = sqrt(x^2+y^2)
    th = atan(y,x) 
    vmax = 20. 
    return min(t,1)*Vec{3}((-r*vmax*sin(th), r*vmax*cos(th), 0.0)) 
end

function assemble_mass_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    # Allocate a buffer for the local matrix and some helpers, together with the assembler.
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Mₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # It follows the assembly loop as explained in the basic tutorials.
    mass_assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Mₑ, 0)
        Ferrite.reinit!(cellvalues_v, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)
            # Remember that we assemble a vector mass term, hence the dot product.
            # There is only one time derivative on the left hand side, so only one mass block is non-zero.
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Mₑ[BlockIndex((v▄, v▄), (i, j))] += φᵢ ⋅ φⱼ * dΩ
                end
            end
        end
        assemble!(mass_assembler, celldofs(cell), Mₑ)
    end

    return M
end;

function assemble_stokes_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, ν, K::SparseMatrixCSC, dh::DofHandler)
    # Again, some buffers and helpers
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Kₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # Assembly loop
    stiffness_assembler = start_assemble(K)
    for cell in CellIterator(dh)
        # Don't forget to initialize everything
        fill!(Kₑ, 0)

        Ferrite.reinit!(cellvalues_v, cell)
        Ferrite.reinit!(cellvalues_p, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)

            for i in 1:n_basefuncs_v
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    ∇φⱼ = shape_gradient(cellvalues_v, q_point, j)
                    Kₑ[BlockIndex((v▄, v▄), (i, j))] -= ν * ∇φᵢ ⊡ ∇φⱼ * dΩ
                end
            end

            for j in 1:n_basefuncs_p
                ψ = shape_value(cellvalues_p, q_point, j)
                for i in 1:n_basefuncs_v
                    divφ = shape_divergence(cellvalues_v, q_point, i)
                    Kₑ[BlockIndex((v▄, p▄), (i, j))] += (divφ * ψ) * dΩ
                    Kₑ[BlockIndex((p▄, v▄), (j, i))] += (ψ * divφ) * dΩ
                end
            end
        end

        # Assemble `Kₑ` into the Stokes matrix `K`.
        assemble!(stiffness_assembler, celldofs(cell), Kₑ)
    end
    return K
end;

function setup_mean_constraint(dh, fvp)
    assembler = Ferrite.COOAssembler()
    # All external boundaries
    set = union(
        getfacetset(dh.grid, "lid"),
        getfacetset(dh.grid, "wall"),
    )
    # Allocate buffers
    range_p = dof_range(dh, :p)
    element_dofs = zeros(Int, ndofs_per_cell(dh))
    element_dofs_p = view(element_dofs, range_p)
    element_coords = zeros(Vec{3}, 4) # assuming mesh with tetrahedra only 
    Ce = zeros(1, length(range_p)) # Local constraint matrix (only 1 row)
    # Loop over all the boundaries
    for (ci, fi) in set
        Ce .= 0
        getcoordinates!(element_coords, dh.grid, ci)
        Ferrite.reinit!(fvp, element_coords, fi)
        celldofs!(element_dofs, dh, ci)
        for qp in 1:getnquadpoints(fvp)
            dΓ = getdetJdV(fvp, qp)
            for i in 1:getnbasefunctions(fvp)
                Ce[1, i] += shape_value(fvp, qp, i) * dΓ
            end
        end
        # Assemble to row 1
        assemble!(assembler, [1], element_dofs_p, Ce)
    end
    C, _ = finish_assemble(assembler)
    # Create an AffineConstraint from the C-matrix
    _, J, V = findnz(C)
    _, constrained_dof_idx = findmax(abs2, V)
    constrained_dof = J[constrained_dof_idx]
    V ./= V[constrained_dof_idx]
    mean_value_constraint = AffineConstraint(
        constrained_dof,
        Pair{Int,Float64}[J[i] => -V[i] for i in 1:length(J) if J[i] != constrained_dof],
        0.0,
    )

    return mean_value_constraint
end

setup_mean_constraint (generic function with 1 method)

## Section 4: Set-up and Assembly

In [44]:
dim = 3 
degree = 1

# Interpolations
ip_v = Lagrange{RefTetrahedron,degree+1}() ^ 3 # quadratic for 3 velocity components 
ip_p = Lagrange{RefTetrahedron,degree}()       # linear for scalar pressure 

dh = DofHandler(grid)

add!(dh, :v, ip_v)
add!(dh, :p, ip_p)

close!(dh);

qr = QuadratureRule{RefTetrahedron}(2)
ipg = Lagrange{RefTetrahedron,1}() # linear geometric interpolation
cellvalues_v = CellValues(qr, ip_v, ipg) # observe three arguments - need to document 
cellvalues_p = CellValues(qr, ip_p, ipg) # observe three arguments - need to document
qr_facet = FacetQuadratureRule{RefTetrahedron}(2)
fvp = FacetValues(qr_facet, ip_p, ipg) # required for pressure constraint 

# Boundary conditions 
ch = ConstraintHandler(dh)

# Boundary conditions part (1/3): Dirichlet BC for the velocity at the top lid 
lid = getfacetset(dh.grid, "lid")
dbc1 = Dirichlet(:v, lid, (x, t) -> lid_rotation(x,t) )
add!(ch, dbc1)

# Boundary conditions part (2/3): no slip boundary condition - impose velocity to be zero vector on the walls   
wall = getfacetset(dh.grid, "wall")
dbc2 = Dirichlet(:v, wall, (x, t) -> [0, 0, 0])
add!(ch, dbc2)
    
# Boundary conditions part (3/3): apply pressure constraint
mean_value_constraint = setup_mean_constraint(dh, fvp)
add!(ch, mean_value_constraint)
    
# Finalize
close!(ch)

M = allocate_matrix(dh,ch);
M = assemble_mass_matrix(cellvalues_v, cellvalues_p, M, dh);

K = allocate_matrix(dh,ch);
viscosity = 1. 
K = assemble_stokes_matrix(cellvalues_v, cellvalues_p, viscosity, K, dh);

uinit = zeros(ndofs(dh))
apply!(uinit, ch);

jac_sparsity = sparse(K);

# apply boundary conditions to the mass-matrix 
apply!(M, ch)

## Section 6: ODE System and Jacobian  

In [45]:
# the non-linear terms do not affect the sparsity pattern 
jac_sparsity = sparse(K);

struct RHSparams
    K::SparseMatrixCSC
    ch::ConstraintHandler
    dh::DofHandler
    cellvalues_v::CellValues
    u::Vector
end
p = RHSparams(K, ch, dh, cellvalues_v, copy(uinit))

function ferrite_limiter!(u, _, p, t)
    update!(p.ch, t)
    apply!(u, p.ch)
end

function navierstokes_rhs_element!(dvₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            dvₑ[j] -= v ⋅ ∇v' ⋅ φⱼ * dΩ
        end
    end
end

function navierstokes!(du,u_uc,p::RHSparams,t)

    @unpack K,ch,dh,cellvalues_v,u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    mul!(du, K, u) # du .= K * u

    # nonlinear contribution
    v_range = dof_range(dh, :v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    vₑ = zeros(n_basefuncs)
    duₑ = zeros(n_basefuncs)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]
        vₑ .= @views u[v_celldofs]
        fill!(duₑ, 0.0)
        navierstokes_rhs_element!(duₑ, vₑ, cellvalues_v)
        assemble!(du, v_celldofs, duₑ)
    end
end;

function navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues_v, q_point, i)
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                Jₑ[j, i] -= (φᵢ ⋅ ∇v' + v ⋅ ∇φᵢ') ⋅ φⱼ * dΩ
            end
        end
    end
end

function navierstokes_jac!(J,u_uc,p,t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    # Here we assume that J has exactly the same structure as K by construction
    nonzeros(J) .= nonzeros(K)

    assembler = start_assemble(J; fillzero=false)

    # Assemble variation of the nonlinear term
    n_basefuncs = getnbasefunctions(cellvalues_v)
    Jₑ = zeros(n_basefuncs, n_basefuncs)
    vₑ = zeros(n_basefuncs)
    v_range = dof_range(dh, :v)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]

        vₑ .= @views u[v_celldofs]
        fill!(Jₑ, 0.0)
        navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
        assemble!(assembler, v_celldofs, Jₑ)
    end

    apply!(J, ch)
end;

T = 6.0
Δt₀ = 0.001
Δt_save = 0.1

T = 5.0  
Δt₀ = 0.001
Δt_save = 0.1

rhs = ODEFunction(navierstokes!, mass_matrix=M; jac=navierstokes_jac!, jac_prototype=jac_sparsity)
problem = ODEProblem(rhs, uinit, (0.0,T), p);

### Test the navierstokes_jac() function 

In [46]:
uinit = zeros(ndofs(dh))
apply!(uinit, ch);
p = RHSparams(K, ch, dh, cellvalues_v, copy(uinit))
J = similar(K)
navierstokes_jac!(J,copy(uinit),p,0.)

## Section 7: Time Integration and Post-Processing 

In [47]:
struct FreeDofErrorNorm
    ch::ConstraintHandler
end
(fe_norm::FreeDofErrorNorm)(u::Union{AbstractFloat, Complex}, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

# timestepper = Rodas5P(autodiff=false, step_limiter! = ferrite_limiter!);
timestepper = Rodas5P(autodiff=false) 
    
integrator = init(
    problem, timestepper; initializealg=NoInit(), dt=Δt₀,
    adaptive=false, 
    progress=true, progress_steps=100,
    verbose=true, internalnorm=FreeDofErrorNorm(ch), d_discontinuities=[1.0]
);

pvd = paraview_collection("ns-cavity")
for (step, (u,t)) in enumerate(intervals(integrator))
    VTKGridFile("ns-cavity-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

LoadError: InterruptException:

In [16]:
struct FreeDofErrorNorm
    ch::ConstraintHandler
end
(fe_norm::FreeDofErrorNorm)(u::Union{AbstractFloat, Complex}, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

# timestepper = Rodas5P(autodiff=false, step_limiter! = ferrite_limiter!);
timestepper = Rodas5P(autodiff=false) 
    
integrator = init(
    problem, timestepper; initializealg=NoInit(), dt=Δt₀,
    adaptive=true, abstol=1e-4, reltol=1e-5,
    progress=true, progress_steps=1,
    verbose=true, internalnorm=FreeDofErrorNorm(ch), d_discontinuities=[1.0]
);

pvd = paraview_collection("ns-cavity")
for (step, (u,t)) in enumerate(intervals(integrator))
    VTKGridFile("ns-cavity-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

LoadError: InterruptException: